In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np
from numpy.random import default_rng

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from soap import extract_species_pair_groups
from skcosmo.decomposition import PCovR

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, PredefinedSplit
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer

# Utilities
import functools
import h5py
import json
import itertools
from tempfile import mkdtemp
from shutil import rmtree
from copy import deepcopy
#from tqdm.notebook import tqdm
from tqdm.auto import tqdm
import project_utils as utils
from tools import load_json, save_json

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

# Load train and test splits

In [3]:
# Load SOAP cutoffs
soap_hyperparameters = load_json('../Processed_Data/soap_hyperparameters.json')   
cutoffs = soap_hyperparameters['interaction_cutoff']

In [4]:
# Load train sets for IZA and Deem
iza_train_idxs = np.loadtxt('../Processed_Data/IZA_230/svm_train.idxs', dtype=int)
iza_sort_idxs = np.argsort(iza_train_idxs)
iza_unsort_idxs = np.argsort(iza_sort_idxs)
deem_train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/svm_train.idxs', dtype=int)

In [5]:
# Load cantons for IZA and Deem
iza_cantons = np.loadtxt('../Raw_Data/IZA_230/cantons.dat', usecols=1, dtype=int)
deem_cantons_2 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_2-class.dat', dtype=int)
deem_cantons_4 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_4-class.dat', dtype=int)

In [6]:
# Build set of "master" canton labels
cantons = {}

cantons[4] = np.concatenate((
    iza_cantons[iza_train_idxs], 
    deem_cantons_4[deem_train_idxs]
))

cantons[2] = np.concatenate((
    np.ones(len(iza_train_idxs), dtype=int),
    deem_cantons_2[deem_train_idxs]
))

# Build set of class weights (by sample) for centering and scaling
class_weights = {n_cantons: utils.balanced_class_weights(cantons[n_cantons]) for n_cantons in (2, 4)}

In [7]:
# Load dummy Deem cantons to test the "null" case
dummy_cantons = {}
dummy_cantons[2] = np.loadtxt('../Processed_Data/DEEM_330k/Data/dummy_cantons_2-class.dat', dtype=int)
dummy_cantons[2] = dummy_cantons[2][deem_train_idxs]
dummy_cantons[4] = np.loadtxt('../Processed_Data/DEEM_330k/Data/dummy_cantons_4-class.dat', dtype=int)
dummy_cantons[4] = dummy_cantons[4][deem_train_idxs]

# Build set of dummy class weights (by sample) for centering and scaling
dummy_class_weights = {n_cantons: utils.balanced_class_weights(dummy_cantons[n_cantons]) for n_cantons in (2, 4)}

# Model setup

In [8]:
model_dir = '../Processed_Data/Models'

deem_name = 'DEEM_330k'
iza_name = 'IZA_230'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

In [9]:
# CV splits
n_splits = 2

y_scaler_parameters = dict(featurewise=False)

pcovr_parameters = dict(n_components=4)
ridge_parameters = dict(fit_intercept=False, tol=1.0E-12)

pcovr_parameter_grid = dict(
    pcovr__regressor__mixing=np.linspace(0.0, 1.0, 3),
    pcovr__regressor__regressor__alpha=np.logspace(-10, 0, 3)
)

ridge_parameter_grid = dict(ridge__regressor__alpha=np.logspace(-10, 0, 3))

# PCovR

In [10]:
# Load SOAPs
iza_file = f'{iza_dir}/6.0/soaps_power_full_avg_nonorm.hdf5'
idxs_sort = np.argsort(iza_train_idxs)
idxs_rev = np.argsort(idxs_sort)
iza_soaps = utils.load_hdf5(iza_file, indices=iza_train_idxs[idxs_sort])
iza_soaps = iza_soaps[idxs_rev]
n_iza_soaps = iza_soaps.shape[0]
# n_train_iza = 50
n_train_iza = n_iza_soaps // 2
# n_train_iza = n_iza_soaps - 2
n_test_iza = n_iza_soaps - n_train_iza

deem_file = f'{deem_dir}/6.0/soaps_power_full_avg_nonorm.hdf5'
deem_soaps = utils.load_hdf5(deem_file, indices=deem_train_idxs)
n_deem_soaps = deem_soaps.shape[0]
# n_train_deem = 5000
n_train_deem = n_deem_soaps // 2
# n_train_deem = n_deem_soaps - 2
n_test_deem = n_deem_soaps - n_train_deem

deem_idxs = np.arange(0, n_deem_soaps)
np.random.shuffle(deem_idxs)
deem_train_idxs2, deem_test_idxs2 = np.split(deem_idxs, [n_train_deem])

iza_idxs = np.arange(0, n_iza_soaps)
np.random.shuffle(iza_idxs)
iza_train_idxs2, iza_test_idxs2 = np.split(iza_idxs, [n_train_iza])

train_soaps = np.vstack((iza_soaps[iza_train_idxs2], deem_soaps[deem_train_idxs2]))
test_soaps = np.vstack((iza_soaps[iza_test_idxs2], deem_soaps[deem_test_idxs2]))

train_cantons = {}
test_cantons = {}

train_cantons[2] = np.concatenate((
    np.ones(n_train_iza, dtype=int),
    np.ones(n_train_deem, dtype=int) * 2
))

test_cantons[2] = np.concatenate((
    np.ones(n_test_iza, dtype=int),
    np.ones(n_test_deem, dtype=int) * 2
))

train_cantons[4] = np.concatenate((
    iza_cantons[iza_train_idxs][iza_train_idxs2],
    np.ones(n_train_deem, dtype=int) * 4
))

test_cantons[4] = np.concatenate((
    iza_cantons[iza_train_idxs][iza_test_idxs2],
    np.ones(n_test_deem, dtype=int) * 4
))

class_weights = {}
class_weights[2] = utils.balanced_class_weights(train_cantons[2])
class_weights[4] = utils.balanced_class_weights(train_cantons[4])

In [11]:
n_cantons = 2
if n_cantons == 2:
    col_slice = slice(0, 1)
else:
    col_slice = slice(0, n_cantons)

In [13]:
# Load decision functions
iza_dfs = np.loadtxt(f'{iza_dir}/6.0/LSVC/{n_cantons}-Class/Power/OO+OSi+SiSi/svc_structure_dfs.dat')
deem_dfs = np.loadtxt(f'{deem_dir}/6.0/LSVC/{n_cantons}-Class/Power/OO+OSi+SiSi/svc_structure_dfs.dat')

train_dfs = np.concatenate((iza_dfs[iza_train_idxs][iza_train_idxs2], deem_dfs[deem_train_idxs][deem_train_idxs2]))
test_dfs = np.concatenate((iza_dfs[iza_train_idxs][iza_test_idxs2], deem_dfs[deem_train_idxs][deem_test_idxs2]))

# Single point

## Ridge

In [46]:
ridge_pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('ridge', TransformedTargetRegressor(
            regressor=Ridge(alpha=1.0E-6, **ridge_parameters),
            transformer=utils.StandardNormScaler(**y_scaler_parameters)
        ))
    ],
)

ridge_pipeline.fit(train_soaps, train_dfs)
ridge_pipeline.predict(test_soaps)

array([-1.59229871, -1.33505447, -2.87091946, ...,  2.49580407,
        2.01959786,  2.66440218])

## PCovR

In [47]:
pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('pcovr', TransformedTargetRegressor(
            regressor=PCovR(
                **pcovr_parameters, 
                mixing=0.0,
                regressor=Ridge(alpha=1.0E-6, **ridge_parameters),
            ),
            transformer=utils.StandardNormScaler(**y_scaler_parameters)
        ))
    ],
)

pipeline.fit(train_soaps, train_dfs)
pipeline.predict(test_soaps)

array([-1.59229874, -1.33505452, -2.87091946, ...,  2.49580409,
        2.01959786,  2.66440219])

# CV

## Ridge

In [15]:
pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('ridge', TransformedTargetRegressor(
            regressor=Ridge(**ridge_parameters),
            transformer=utils.StandardNormScaler(**y_scaler_parameters)
        ))
    ],
)

# IZA + DEEM classification
gscv = GridSearchCV(
    pipeline, ridge_parameter_grid,
    scoring='neg_root_mean_squared_error',
    cv=KFold(n_splits=n_splits, shuffle=True, random_state=0),
    refit=False, return_train_score=True, error_score='raise',
    verbose=2
)

gscv.fit(train_soaps, train_dfs)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] END ......................ridge__regressor__alpha=1e-10; total time=   0.4s
[CV] END ......................ridge__regressor__alpha=1e-10; total time=   0.3s
[CV] END ......................ridge__regressor__alpha=1e-05; total time=   0.4s
[CV] END ......................ridge__regressor__alpha=1e-05; total time=   0.3s
[CV] END ........................ridge__regressor__alpha=1.0; total time=   0.4s
[CV] END ........................ridge__regressor__alpha=1.0; total time=   0.3s


GridSearchCV(cv=KFold(n_splits=2, random_state=0, shuffle=True),
             error_score='raise',
             estimator=Pipeline(steps=[('norm_scaler', StandardNormScaler()),
                                       ('ridge',
                                        TransformedTargetRegressor(regressor=Ridge(fit_intercept=False,
                                                                                   tol=1e-12),
                                                                   transformer=StandardNormScaler()))]),
             param_grid={'ridge__regressor__alpha': array([1.e-10, 1.e-05, 1.e+00])},
             refit=False, return_train_score=True,
             scoring='neg_root_mean_squared_error', verbose=2)

In [16]:
gscv.cv_results_

{'mean_fit_time': array([0.31750762, 0.31362426, 0.31510985]),
 'std_fit_time': array([0.0026058 , 0.0097183 , 0.01028287]),
 'mean_score_time': array([0.03173792, 0.03064442, 0.03054595]),
 'std_score_time': array([0.0053283 , 0.00053525, 0.0005765 ]),
 'param_ridge__regressor__alpha': masked_array(data=[1e-10, 1e-05, 1.0],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'ridge__regressor__alpha': 1e-10},
  {'ridge__regressor__alpha': 1e-05},
  {'ridge__regressor__alpha': 1.0}],
 'split0_test_score': array([-1.48292738e-10, -8.75719000e-06, -5.36717411e-02]),
 'split1_test_score': array([-7.09144718e-10, -9.60682552e-06, -5.77138167e-02]),
 'mean_test_score': array([-4.28718728e-10, -9.18200776e-06, -5.56927789e-02]),
 'std_test_score': array([2.80425990e-10, 4.24817760e-07, 2.02103782e-03]),
 'rank_test_score': array([1, 2, 3], dtype=int32),
 'split0_train_score': array([-4.78323105e-11, -3.57345093e-06, -5.22047931e-02]),
 '

## PCovR

In [17]:
pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('pcovr', TransformedTargetRegressor(
            regressor=PCovR(**pcovr_parameters),
            transformer=utils.StandardNormScaler(**y_scaler_parameters)
        ))
    ],
)

# IZA + DEEM classification
gscv = GridSearchCV(
    pipeline, pcovr_parameter_grid,
    scoring=utils.pcovr_score,
    cv=KFold(n_splits=n_splits, shuffle=True, random_state=0),
    refit=False, return_train_score=True, error_score='raise',
    verbose=2
)

gscv.fit(train_soaps, train_dfs)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-10; total time=   6.0s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-10; total time=   6.1s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-05; total time=   6.0s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-05; total time=   6.1s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1.0; total time=   6.0s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1.0; total time=   6.0s
[CV] END pcovr__regressor__mixing=0.5, pcovr__regressor__regressor__alpha=1e-10; total time=   6.1s
[CV] END pcovr__regressor__mixing=0.5, pcovr__regressor__regressor__alpha=1e-10; total time=   6.3s
[CV] END pcovr__regressor__mixing=0.5, pcovr__regressor__regressor__alpha=1e-05; total time=   6.1s
[CV] END pcovr__regressor__mixing=0.5, pcovr

GridSearchCV(cv=KFold(n_splits=2, random_state=0, shuffle=True),
             error_score='raise',
             estimator=Pipeline(steps=[('norm_scaler', StandardNormScaler()),
                                       ('pcovr',
                                        TransformedTargetRegressor(regressor=PCovR(n_components=4,
                                                                                   regressor=Ridge(alpha=1e-06,
                                                                                                   fit_intercept=False,
                                                                                                   tol=1e-12)),
                                                                   transformer=StandardNormScaler()))]),
             param_grid={'pcovr__regressor__mixing': array([0. , 0.5, 1. ]),
                         'pcovr__regressor__regressor__alpha': array([1.e-10, 1.e-05, 1.e+00])},
             refit=False, return_train_score=True,
 

In [18]:
gscv.cv_results_

{'mean_fit_time': array([5.90916049, 5.91694891, 5.89721668, 6.07170415, 5.96476841,
        5.98468328, 5.98111844, 5.97968423, 5.99518287]),
 'std_fit_time': array([0.02554142, 0.01008832, 0.00077999, 0.08218575, 0.00325465,
        0.01145029, 0.00961399, 0.01055801, 0.04881632]),
 'mean_score_time': array([0.14291453, 0.12813497, 0.12724221, 0.12556481, 0.13017416,
        0.13475168, 0.11060739, 0.12984157, 0.13106203]),
 'std_score_time': array([0.0011766 , 0.01666141, 0.01502335, 0.01584291, 0.01586008,
        0.00801361, 0.00029945, 0.01322436, 0.01225948]),
 'param_pcovr__regressor__mixing': masked_array(data=[0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_pcovr__regressor__regressor__alpha': masked_array(data=[1e-10, 1e-05, 1.0, 1e-10, 1e-05, 1.0, 1e-10, 1e-05,
                    1.0],
              mask=[False, F

# CV with oversampling

In [19]:
class DataPrinter():
    def fit(self, X, y=None):
        print(X)
        if y is not None:
            print(y)
            
        return self
            
    def transform(self, X):
        print(X)
        return X

## Ridge

In [30]:
y_pipeline = Pipeline(
    [
#         ('drop_features', utils.ColumnTransformerInverse([('drop_features', 'passthrough', col_slice)])),
        ('drop_features', utils.ColumnTransformerInverse(
            [('drop_features', 'drop', -1)],
            remainder='passthrough'
        )),
        ('norm_scaler', utils.StandardNormScaler(**y_scaler_parameters))
    ]
)

pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
#         ('print', DataPrinter()),
        ('ridge', TransformedTargetRegressor(
            regressor=Ridge(**ridge_parameters),
            transformer=y_pipeline,
            check_inverse=False
        ))
    ],
)

# IZA + DEEM classification
gscv = GridSearchCV(
    pipeline, ridge_parameter_grid,
    scoring=make_scorer(
        utils.class_balanced_metric_score,
        scorer=mean_squared_error,
        class_col=-1,
        squared=False,
        greater_is_better=False
    ),
    cv=utils.ReplicatedStratifiedKFold(n_splits=n_splits, stratify_col=-1, shuffle=True, random_state=0),
    refit=True, return_train_score=True, error_score='raise',
    verbose=2
)

gscv.fit(train_soaps, np.column_stack((train_dfs, train_cantons[n_cantons])))

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] END ......................ridge__regressor__alpha=1e-10; total time=   0.5s
[CV] END ......................ridge__regressor__alpha=1e-10; total time=   0.5s
[CV] END ......................ridge__regressor__alpha=1e-05; total time=   0.5s
[CV] END ......................ridge__regressor__alpha=1e-05; total time=   0.5s
[CV] END ........................ridge__regressor__alpha=1.0; total time=   0.5s
[CV] END ........................ridge__regressor__alpha=1.0; total time=   0.5s


GridSearchCV(cv=ReplicatedStratifiedKFold(n_splits=2, random_state=0, shuffle=True,
             stratify_col=-1),
             error_score='raise',
             estimator=Pipeline(steps=[('norm_scaler', StandardNormScaler()),
                                       ('ridge',
                                        TransformedTargetRegressor(check_inverse=False,
                                                                   regressor=Ridge(fit_intercept=False,
                                                                                   tol=1e-12),
                                                                   transformer=Pipeline(steps=[('drop_features',
                                                                                                ColumnTransformerInve...[('drop_features',
                                                                                                                                        'passthrough',
                                  

In [31]:
gscv.predict(test_soaps)

array([[-1.38715866],
       [-1.09143563],
       [ 0.36412421],
       ...,
       [ 0.85145064],
       [ 6.75896238],
       [ 2.24166333]])

## PCovR

In [28]:
y_pipeline = Pipeline(
    [
#         ('drop_features', utils.ColumnTransformerInverse([('drop_features', 'passthrough', col_slice)])),
        ('drop_features', utils.ColumnTransformerInverse(
            [('drop_features', 'drop', -1)], remainder='passthrough'
        )),
        ('norm_scaler', utils.StandardNormScaler(**y_scaler_parameters))
    ]
)

pipeline = Pipeline(
    [
#         ('print', DataPrinter()),
        ('norm_scaler', utils.StandardNormScaler()),
        ('pcovr', TransformedTargetRegressor(
#             regressor=PCovR(mixing=0.0, **pcovr_parameters),
            regressor=PCovR(**pcovr_parameters),
            transformer=y_pipeline,
            check_inverse=False
        ))
    ],
)

# IZA + DEEM classification
gscv = GridSearchCV(
    pipeline, pcovr_parameter_grid,
    scoring={
        'pcovr': functools.partial(utils.class_balanced_pcovr_score, class_col=-1),
        'rmse': make_scorer(
            utils.class_balanced_metric_score,
            scorer=mean_squared_error,
            class_col=-1,
            squared=False,
            greater_is_better=False
        )
    },
    cv=utils.ReplicatedStratifiedKFold(n_splits=n_splits, stratify_col=-1, shuffle=True, random_state=0),
    refit='pcovr', return_train_score=True, error_score='raise',
    verbose=2
)

gscv.fit(train_soaps, np.column_stack((train_dfs, train_cantons[n_cantons])))

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-10; total time=   6.5s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-10; total time=   6.3s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-05; total time=   6.5s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1e-05; total time=   6.3s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1.0; total time=   6.3s
[CV] END pcovr__regressor__mixing=0.0, pcovr__regressor__regressor__alpha=1.0; total time=   6.3s
[CV] END pcovr__regressor__mixing=0.5, pcovr__regressor__regressor__alpha=1e-10; total time=   6.5s
[CV] END pcovr__regressor__mixing=0.5, pcovr__regressor__regressor__alpha=1e-10; total time=   6.4s
[CV] END pcovr__regressor__mixing=0.5, pcovr__regressor__regressor__alpha=1e-05; total time=   6.5s
[CV] END pcovr__regressor__mixing=0.5, pcovr

GridSearchCV(cv=ReplicatedStratifiedKFold(n_splits=2, random_state=0, shuffle=True,
             stratify_col=-1),
             error_score='raise',
             estimator=Pipeline(steps=[('norm_scaler', StandardNormScaler()),
                                       ('pcovr',
                                        TransformedTargetRegressor(check_inverse=False,
                                                                   regressor=PCovR(n_components=4,
                                                                                   regressor=Ridge(alpha=1e-06,
                                                                                                   fit_intercept=False,
                                                                                                   tol=1e-12)),
                                                                   transformer=Pipeline(s...
                         'pcovr__regressor__regressor__alpha': array([1.e-10, 1.e-05, 1.e+00])},
   

In [29]:
gscv.predict(test_soaps)

array([[-1.35488922],
       [-0.9396629 ],
       [ 0.35352605],
       ...,
       [ 0.88023965],
       [ 6.79922626],
       [ 2.25907879]])

In [24]:
gscv.cv_results_

{'mean_fit_time': array([6.10000634, 6.0878762 , 6.1032238 , 6.3533957 , 6.29492998,
        6.29024899, 6.29276741, 6.29065943, 6.3384794 ]),
 'std_fit_time': array([0.01149154, 0.01272595, 0.00660896, 0.07314229, 0.00877452,
        0.0013479 , 0.00630844, 0.00952983, 0.03946459]),
 'mean_score_time': array([0.16890574, 0.16970551, 0.18906629, 0.18010783, 0.18232131,
        0.17057621, 0.18263209, 0.18077219, 0.17820716]),
 'std_score_time': array([0.00018692, 0.00112069, 0.00234258, 0.01030493, 0.01221991,
        0.00011671, 0.01323068, 0.01078165, 0.00802684]),
 'param_pcovr__regressor__mixing': masked_array(data=[0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_pcovr__regressor__regressor__alpha': masked_array(data=[1e-10, 1e-05, 1.0, 1e-10, 1e-05, 1.0, 1e-10, 1e-05,
                    1.0],
              mask=[False, F

# Sample weight pipeline

In [ ]:
from sklearn.svm import LinearSVC

## Single point

In [ ]:
pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('svc', LinearSVC(class_weight='balanced', max_iter=10000, dual=False))
    ],
)

wts = utils.balanced_class_weights(train_cantons[n_cantons])
fit_params = {'norm_scaler__sample_weight': wts}
pipeline.fit(train_soaps, train_cantons[n_cantons], **fit_params)
pipeline.decision_function(test_soaps)

In [ ]:
pipeline = utils.ClassBalancedPipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('svc', LinearSVC(class_weight='balanced', max_iter=10000, dual=False))
    ],
)

pipeline.fit(train_soaps, train_cantons[n_cantons], keys=['norm_scaler__sample_weight'])
pipeline.decision_function(test_soaps)

## CV

In [ ]:
svc_parameter_grid = {'svc__C': np.logspace(-1, 1, 3)}

In [ ]:
# No weights
pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('svc', LinearSVC(class_weight='balanced', max_iter=10000, dual=False))
    ],
)

gscv = GridSearchCV(
    pipeline, svc_parameter_grid,
    scoring='balanced_accuracy',
    cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0),
    refit=True, return_train_score=True, error_score='raise',
    verbose=2
)

gscv.fit(train_soaps, train_cantons[n_cantons])
gscv.decision_function(test_soaps)

In [ ]:
gscv.cv_results_

In [ ]:
# fit_params weights
pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('svc', LinearSVC(class_weight='balanced', max_iter=10000, dual=False))
    ],
)

gscv = GridSearchCV(
    pipeline, svc_parameter_grid,
    scoring='balanced_accuracy',
    cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0),
    refit=True, return_train_score=True, error_score='raise',
    verbose=2
)

wts = utils.balanced_class_weights(train_cantons[n_cantons])
fit_params = {'norm_scaler__sample_weight': wts}
gscv.fit(train_soaps, train_cantons[n_cantons], **fit_params)
gscv.decision_function(test_soaps)

In [ ]:
gscv.cv_results_

In [ ]:
# ClassBalancedPipeline weights
pipeline = utils.ClassBalancedPipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('svc', LinearSVC(class_weight='balanced', max_iter=10000, dual=False))
    ],
)

gscv = GridSearchCV(
    pipeline, svc_parameter_grid,
    scoring='balanced_accuracy',
    cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0),
    refit=True, return_train_score=True, error_score='raise',
    verbose=2
)

gscv.fit(train_soaps, train_cantons[n_cantons], keys=['norm_scaler__sample_weight'])
gscv.decision_function(test_soaps)

In [ ]:
gscv.cv_results_

# Testing weights

This might work but probably better for the time being to work with replicated samples, despite the computational expense...

In [ ]:
w = utils.balanced_class_weights(train_cantons[n_cantons])
W = np.diagflat(w)
Wsqrt = np.sqrt(W)

In [ ]:
# Weights as they should be
np.trace(Wsqrt @ train_soaps @ train_soaps.T @ Wsqrt)

In [ ]:
# Weighted covariance
train_soaps.T @ W @ train_soaps

In [ ]:
rpt_idxs = repeat_idxs(train_cantons[n_cantons])
replicated_train_soaps = np.repeat(train_soaps, rpt_idxs, axis=0)
replicated_train_cantons = np.repeat(train_cantons[n_cantons], rpt_idxs)

In [ ]:
# "Weights" from replication
np.trace(replicated_train_soaps @ replicated_train_soaps.T) / len(replicated_train_soaps)

In [ ]:
# Replicated covariance
replicated_train_soaps.T @ replicated_train_soaps / len(replicated_train_soaps)

In [ ]:
wv, wu = np.linalg.eigh(Wsqrt @ train_soaps @ train_soaps.T @ Wsqrt)

In [ ]:
rv, ru = np.linalg.eigh(replicated_train_soaps @ replicated_train_soaps.T / len(replicated_train_soaps))

In [ ]:
np.count_nonzero(wv > 1.0E-12), np.count_nonzero(rv  > 1.0E-12)

In [ ]:
unique_labels, label_counts = np.unique(train_cantons[n_cantons], return_counts=True)
split_rpt_idxs = np.split(rpt_idxs, np.cumsum(label_counts))[0:-1]
class_counts = np.array([len(counts) for counts in split_rpt_idxs])
replicated_class_counts = np.array([np.sum(counts) for counts in split_rpt_idxs])
extra_samples = (replicated_class_counts - replicated_class_counts[-1]) / class_counts

labels = train_cantons[n_cantons]
n_samples = len(train_cantons[n_cantons])
n_classes = len(unique_labels)
wr = np.zeros(n_samples)
for ul, lc in zip(unique_labels, label_counts+extra_samples):
    label_weight = (n_samples + np.sum(extra_samples * class_counts)) / (n_classes * lc)
    wr[labels == ul] = label_weight
    
wr /= np.sum(wr)
Wr = np.diagflat(wr)
Wrsqrt = np.sqrt(Wr)

In [ ]:
np.trace(Wrsqrt @ train_soaps @ train_soaps.T @ Wrsqrt)

# Testing RR with replicated samples vs. sample weights

In [ ]:
# Repeats
ridge_pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('ridge', TransformedTargetRegressor(
            regressor=RidgeCV(alphas=np.logspace(-8, 0, 5), fit_intercept=False, cv=2),
            #regressor=Ridge(alpha=1.0E-6, **ridge_parameters),
            transformer=utils.StandardNormScaler(featurewise=True)
        ))
    ],
)

ridge_pipeline.fit(np.repeat(train_soaps, rpt_idxs, axis=0), np.repeat(train_dfs, rpt_idxs, axis=0))
ridge_pipeline.predict(test_soaps)

In [ ]:
rmse = np.zeros(len(unique_labels))
for udx, ul in enumerate(unique_labels):
    rmse[udx] = mean_squared_error(
        test_dfs[test_cantons[n_cantons] == ul], 
        ridge_pipeline.predict(test_soaps[test_cantons[n_cantons] == ul]), 
        squared=False
    )

print(mean_squared_error(test_dfs, ridge_pipeline.predict(test_soaps), squared=False))
print(rmse)
print(np.mean(rmse))

In [ ]:
# Weights
ww = np.ones(len(train_dfs)) / len(train_dfs)
ridge_pipeline = Pipeline(
    [
        ('norm_scaler', utils.StandardNormScaler()),
        ('ridge', TransformedTargetRegressor(
            regressor=RidgeCV(alphas=np.logspace(-8, 0, 5), fit_intercept=False, cv=2),
            #regressor=Ridge(alpha=1.0E-6, **ridge_parameters),
            transformer=utils.DataSplitter(
                model=utils.StandardNormScaler(featurewise=True),
                X_cols=slice(0, -1),
                weight_col=-1
            ),
            check_inverse=False
        ))
    ],
)
fit_params = dict(
    norm_scaler__sample_weight=ww,
    ridge__sample_weight=ww
)
ridge_pipeline.fit(train_soaps, np.column_stack((train_dfs, ww)), **fit_params)
ridge_pipeline.predict(test_soaps)

In [ ]:
rmse = np.zeros(len(unique_labels))
for udx, ul in enumerate(unique_labels):
    rmse[udx] = mean_squared_error(
        test_dfs[test_cantons[n_cantons] == ul], 
        ridge_pipeline.predict(test_soaps[test_cantons[n_cantons] == ul]), 
        squared=False
    )
    
print(mean_squared_error(test_dfs, ridge_pipeline.predict(test_soaps).squeeze(), squared=False))
print(rmse)
print(np.mean(rmse))

# Testing PCA with replicated samples vs. sample weights